This notebook demonstrates how to parse genome download from [PATRIC](https://www.patricbrc.org/), including reading, CD searching, and printing.


In [3]:
import os
import time

from Bio import SeqIO
from Bio.Blast.Applications import NcbirpsblastCommandline

# path to CDD (conserved domain database) and related info
CDD_PATH = os.path.abspath('../data/interim/CDD/Cdd')
RPSBLAST_KWARGS = {
    'db': CDD_PATH,
    'seg': 'no',
    'comp_based_stats': '1',
    'evalue': 0.01,
    'outfmt': 5,
}

# path to all the genomes (downloaded from PATRIC directly)
GENOME_PARENT_PATH = './genome_parsing_examples'


/Users/xduan7/projects/bioseq-learning/data/interim/CDD/cdd


[PATRIC USER GUIDES/FTP](https://docs.patricbrc.org/user_guides/ftp.html):
> Each genome directory provides the following data files for PATRIC and RefSeq annotations (when available).
- fna: FASTA contig sequences
- faa: FASTA protein sequence file
- features.tab: All genomic features and related information in tab-delimited format
- ffn: FASTA nucleotide sequences for genomic features, i.e. genes, RNAs, and other misc features
- frn: FASTA nucleotide sequences for RNAs
- gff: Genome annotations in GFF file format
- pathway.tab: Metabolic pathway assignments in tab-delimited format
- spgene.tab: Specialty gene assignments (i.e. AMR genes, virulance factors, essential genes, etc) in tab-delimited format
- subsystem.tab: Subsystem assignments in tab-delimited format



In [2]:
# get all the genome names (PATRIC ID)
genome_names = next(os.walk(GENOME_PARENT_PATH))[1]


# loop over all the genomes on parent path and perform parsing ...
for _genome_name in genome_names:

    _genome_path = os.path.join(GENOME_PARENT_PATH, _genome_name)

    # get the nucleotide sequences (whole contig and the one with features only)
    _cntg_path = os.path.join(_genome_path, _genome_name + '.fna')
    _cntg_records = list(SeqIO.parse(_cntg_path, 'fasta'))
    _feat_path = os.path.join(_genome_path, _genome_name + '.PATRIC.ffn')
    _feat_records = list(SeqIO.parse(_feat_path, 'fasta'))

    print(f'Genome {_genome_name} contains '
          f'{len(_cntg_records)} contig(s) and '
          f'{len(_feat_records)} features (genes, RNAs, etc.)')

    # perform CD search for the contigs with measured time
    _start_time = time.time()
    _cntg_rpsblast_cmd = NcbirpsblastCommandline(query=_cntg_path, **RPSBLAST_KWARGS)
    _cntg_rpsblast_xml_result, _cntg_rpsblast_cmd_error_msg = _cntg_rpsblast_cmd()
    print(time.time() - _start_time)
    with open(os.path.join(GENOME_PARENT_PATH, 'rpsblast_contig_' + _genome_name + '.xml'), 'w+') as f:
        f.write(_cntg_rpsblast_xml_result)

    # perform CD search for the features only with measure time
    _start_time = time.time()
    _feat_rpsblast_cmd = NcbirpsblastCommandline(query=_feat_path, **RPSBLAST_KWARGS)
    _feat_rpsblast_xml_result, _feat_rpsblast_cmd_error_msg = _feat_rpsblast_cmd()
    print(time.time() - _start_time)
    with open(os.path.join(GENOME_PARENT_PATH, 'rpsblast_feature_' + _genome_name + '.xml'), 'w+') as f:
        f.write(_feat_rpsblast_xml_result)

    # TODO: make sure that the two CD searches have the same results ...


Genome 83333.84 contains 1 contig(s) and 4357 features (genes, RNAs, etc.)


KeyboardInterrupt: 